# Armani SageMaker instance 

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
# from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the eu-north-1 region. You will use the 669576153137.dkr.ecr.eu-north-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
bucket_name = 'cestino-s3'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


The part until now is equal to the [tutorial](https://aws.amazon.com/it/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/).

In [3]:
try:
  urllib.request.urlretrieve("https://leonardo-michelazzo.github.io/LLama-AI/categorizzatore/Armani.csv", "./products.csv")
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./products.csv') # Changed this
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


Now that we have loaded into model_data the CSV...

In [4]:
print(model_data) # ...and after a bit of debugging...
print(type(model_data)) # ...and compensation of the absurd leak of documentation...

        0          1                           2   
0   donna    giacche                         blu  \
1   donna  pantaloni  grigioscuro,ghiaccio-e-blu   
2   donna  pantaloni                    fantasia   
3   donna   sneakers                bianco-e-blu   
4   donna      abiti                  nero-e-blu   
5   donna    giacche                    fantasia   
6   donna    sandali               argento-e-oro   
7   donna    sandali                        nero   
8   donna    sandali              nero-e-marrone   
9   donna    sandali                        nero   
10  donna    giacche                     blunavy   
11  donna  pantaloni              blunavy-e-nero   
12  donna     scarpe                        nero   
13  donna      abiti                        rosa   
14  donna     scarpe                 nude-e-nero   
15  donna     scarpe                 nude-e-nero   
16  donna      abiti                  blu-e-nero   
17  donna    sandali                         oro   
18  donna   

...we start splitting data into the data for training (in this case 70% of the total) an data for testing (the rest).

In [5]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(35, 10) (15, 10)


We should first get what [random_state](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) is.
Well, it's just a randomizer I guess.

In [6]:
print(train_data)
print(test_data)

        0          1                         2                              3   
23   uomo    giacche                       blu                   misto-cotone  \
32   uomo  pantaloni                  fantasia                          cupro   
16  donna      abiti                blu-e-nero                  misto-viscosa   
42   uomo      abiti            nero-e-blunavy                   lana-vergine   
30   uomo  pantaloni                     beige                   lana-vergine   
14  donna     scarpe               nude-e-nero                          pelle   
43   uomo    giacche                     beige  viscosa,-poliammide-e-elastan   
17  donna    sandali                       oro                 viscosa-e-seta   
40   uomo      abiti                      nero                   lana-vergine   
15  donna     scarpe               nude-e-nero                          pelle   
20  donna    giacche                       blu                           lino   
44   uomo    giacche        

In [7]:
train_data.to_csv(
    'train.csv',
    index=False,
    header=True
)
boto3.Session().resource('s3').Bucket(bucket_name).Object(
    os.path.join(
        prefix, 'train/train.csv'
    )
).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='csv'
)

In [8]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(
    xgboost_container,role, instance_count=1,
    instance_type='ml.m5.4xlarge',
    output_path=f's3://{bucket_name}/{prefix}/output',
    sagemaker_session=sess
)
xgb.set_hyperparameters(
    max_depth=5,eta=0.2,gamma=4,
    min_child_weight=6,subsample=0.8,
    silent=0,objective='binary:logistic',
    num_round=100
)

In [9]:
print(s3_input_train)

In [10]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-06-23-09-35-32-575


2023-06-23 09:35:32 Starting - Starting the training job...
2023-06-23 09:35:59 Starting - Preparing the instances for training......
2023-06-23 09:36:56 Downloading - Downloading input data......
2023-06-23 09:37:57 Training - Training image download completed. Training in progress.
2023-06-23 09:37:57 Uploading - Uploading generated training modelArguments: train
[2023-06-23:09:37:51:INFO] Running standalone xgboost training.
[2023-06-23:09:37:51:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-06-23:09:37:51:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 55609.43mb
[2023-06-23:09:37:51:INFO] Determined delimiter of CSV input is ','
[09:37:51] S3DistributionType set as FullyReplicated
[09:37:51] 47x19 matrix with 529 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[09:37:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[0]#011train-error